In [23]:
'''
from google.colab import files
uploaded = files.upload()
'''


'\nfrom google.colab import files\nuploaded = files.upload()\n'

In [80]:
import csv
import re

flavour_dict = {}
reversed_flavour_dict = {}

with open('item-flavors.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    count = 0
    for row in spamreader:
        flavour_dict['flavour' + str(count)] = [x.strip().lower() for x in row]   
        flavour_dict['flavour' + str(count)].append('flavour' + str(count))
        count+=1
        
for k,v in flavour_dict.items():
    for flav in v:
        reversed_flavour_dict[flav] = k
        
size_dict = {}
reversed_size_dict = {}

with open('item-size.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')  
    count = 0
    for row in spamreader:
        size_dict['size' + str(count)] = [x for x in row]
        size_dict['size' + str(count)].append('size' + str(count))
        count+=1

for k,v in size_dict.items():
    for size in v:
        reversed_size_dict[size] = k

qty_dict = {'a':1,'one':1,'two':2,'three':3,'four':4,'five':5,'six':6,'seven':7,'eight':8,'nine':9,
           'ten':10}
quantifiers = {'all':1,'both':2}
action_words = {'make': 0}

In [145]:
def gen_order(ord_l, flavour_l = None):
    is_success = 0
    size_l = []
    if flavour_l is None:
        flavour_l = []
    qty_l = []
    all_flavours = []
    ord_dict = {}
    iter = 0
    for word in ord_l:
        if word in reversed_flavour_dict:
            all_flavours.append(word)
    while(iter<len(ord_l)):        
        word = ord_l[iter]
        if word in reversed_flavour_dict:
            flavour_l.append(word)
        elif word in reversed_size_dict:
            size_l.append(word)
        elif word in qty_dict:
            qty_l.append(word)
        elif word in quantifiers:
            if word == 'both':
                next_word = ord_l[iter+1]
                if next_word in reversed_size_dict:
                    for i in range(quantifiers['both']):
                        size_l.append(next_word)
                    iter+=1
                elif len(size_l)*len(qty_l) == 1 and len(flavour_l) == 0:
                    if next_word in reversed_flavour_dict:
                        next_next_word = ord_l[iter+2]
                        if next_next_word in reversed_flavour_dict:
                            flavour_l.append(next_word)
                            flavour_l.append(next_next_word)
                            size_l.append(size_l[0])
                            qty_l.append(qty_l[0])
                            iter+=2
            if word == 'all':
                next_word = ord_l[iter+1]
                if next_word in qty_dict:
                    next_next_word = ord_l[iter+1]
                    if next_next_word in reversed_size_dict:
                        for i in range(qty_dict[next_word]):
                            size_l.append(next_next_word)
                        iter+=2
                elif next_word in size_dict:
                    if len(flavour_l)!= len(qty_l):
                        print("Sorry I didn't get you, please try again.")
                        return all_flavours, 0
                        break
                    else:
                        for i in range(len(flavour_l)):
                            size_l.append(next_word)
                        iter+=1
        elif word in action_words:
            if word == 'make':
                count = 0
                no_active_flavours = len(flavour_l)                   
                active_index = -1
                for subword in ord_l[iter+1:]:
                    if subword in reversed_flavour_dict:
                        active_index = flavour_l.index(subword)
                        flavour_l[0], flavour_l[active_index] =  flavour_l[active_index], flavour_l[0]
                        if len(size_l) > active_index:
                            size_l[0], size_l[active_index] = size_l[active_index], size_l[0]
                        if len(qty_l) > active_index:
                            qty_l[0], qty_l[active_index] = qty_l[active_index], qty_l[0]
                        active_index = 0
                    elif subword in reversed_size_dict:
                        size_l.append(subword)
                    elif subword in qty_dict:
                        qty_l.append(subword)
                    while (len(size_l)*len(flavour_l)*len(qty_l)!= 0):
                        ord_dict[len(ord_dict)] = [flavour_l[0], size_l[0], qty_l[0]]
                        size_l.pop(active_index)
                        flavour_l.pop(active_index)
                        qty_l.pop(active_index)
                        active_index = -1
                    
                    iter+=1
                        
                                                                        
        while (len(size_l)*len(flavour_l)*len(qty_l)!= 0):
            ord_dict[len(ord_dict)] = [flavour_l[0], size_l[0], qty_l[0]]
            size_l.pop(0)
            flavour_l.pop(0)
            qty_l.pop(0)
            
        iter+=1
    if (len(size_l) + len(flavour_l) + len(qty_l)!= 0):
      return all_flavours, 0
    else:
        return ord_dict, 1
        
        
        
def string_preprocess(ord_s):
    
    ord_s = ord_s.strip().lower()
    for k,v in reversed_size_dict.items():
        ord_s = ord_s.replace(k,v)
    
    for k,v in reversed_flavour_dict.items():
        ord_s = ord_s.replace(k,v)
    ord_s = re.sub(r'[^\w\s]','',ord_s)  #Remove Punctuation

    ord_l = ord_s.split(" ")
    ord_temp = []
    for word in ord_l:
        if (word in reversed_flavour_dict or word in reversed_size_dict or word in qty_dict or 
            word in quantifiers or word in action_words):
            ord_temp.append(word)
    
    return ord_temp

def prettify_order(order_dict:dict):
    for order in order_dict:
        order_dict[order][0] = flavour_dict[order_dict[order][0]][0]
        order_dict[order][1] = size_dict[order_dict[order][1]][0]
    return order_dict


    
    
def reask(flavour_l):
    print("We've detected you've asked for " + " ,".join([reversed_flavour_dict[flavour] for flavour in flavour_l])+
                                                        " flavours")
    print("Could you please specify the size and quantities")
    ord_s = "Two Medium and One small"
    ord_l = string_preprocess(ord_s)
    ord_dict, is_success = gen_order(ord_l, flavour_l)
    if (is_success):
        return ord_dict, is_success
    else:
        return {}, 0
        
    
    
    

In [148]:
def main():
    ord_s = "Mango two and anjeer three both small"
    ord_l = string_preprocess(ord_s)
    ord_dict, is_success = gen_order(ord_l, None)
    
    if is_success:
        print(prettify_order(ord_dict))
    else:
        ord_dict, is_success = reask(list(set(ord_dict)))
        if is_success:
            print(prettify_order(ord_dict))
        else:
            print("Sorry we couldn't understand your order")
    

In [149]:
'''

Cases that can be handled so far

I want one mango and one anjeer both small
I want one mango and one anjeer, all small and one large fruit
I want one mango medium and one anjeer large
I want one small mango, one anjeer and one fruit both small and one small malai
Mango two and anjeer three both small
Two mango and one anjeer, make mango large but anjeer small
Two mango and one anjeer, make anjeer large but mango small

'''

'\n\nCases that can be handled so far\n\nI want one mango and one anjeer both small\nI want one mango and one anjeer, all small and one large fruit\nI want one mango medium and one anjeer large\nI want one small mango, one anjeer and one fruit both small and one small malai\nMango two and anjeer three both small\nTwo mango and one anjeer, make mango large but anjeer small\n\n'

In [150]:
main()

{0: ['mango', 'single scoop', 'two'], 1: ['anjeer', 'single scoop', 'three']}
